In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import torch
from torch.autograd import Variable
import torch.utils.data

from src.data_handling.io_interaction import read_as_pandas_from_disk
from src.utils.helpers import parse_data_for_ml

# Data import and parsing

In [ ]:
data = read_as_pandas_from_disk("./data/training-data.txt")
data_parsed = parse_data_for_ml(data)

In [ ]:
training_set = data_parsed.iloc[:,1:2].values
plt.plot(training_set, label = 'Temperature')
plt.show()

## Smoothing the data

In [ ]:
# Data should be smoothed before training
# We will use a moving average to smooth the data
# We will use a window of x timesteps
moving_average_window = 1
moving_average = np.convolve(training_set[:,0], np.ones(moving_average_window)/moving_average_window, mode='same')
# Bring into same format as training_set (maybe remove later, check with sliding window algo)
moving_average = moving_average.reshape(-1,1)

In [ ]:
# Plot the training set and the moving average on four different points in time
# Increase size of plot
plt.rcParams['figure.figsize'] = [12, 8]
plt.subplot(2, 2, 1)
plt.plot(training_set[0:100], label = 'Temperature')
plt.plot(moving_average[0:100], label = 'Temperature', color='red')
plt.subplot(2, 2, 2)
plt.plot(training_set[1000:1100], label = 'Temperature')
plt.plot(moving_average[1000:1100], label = 'Temperature', color='red')
plt.subplot(2, 2, 3)
plt.plot(training_set[2000:2100], label = 'Temperature')
plt.plot(moving_average[2000:2100], label = 'Temperature', color='red')
plt.subplot(2, 2, 4)
plt.plot(training_set[len(training_set)-500:len(training_set)], label = 'Temperature')
plt.plot(moving_average[len(training_set)-500:len(training_set)], label = 'Temperature', color='red')
plt.show()


In [ ]:
training_set = moving_average

# Sliding Windows

In [ ]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-2*seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length:i+seq_length*2].reshape(-1, 1)
        x.append(_x)
        y.append(_y.reshape(1,-1)[0])

    return np.array(x), np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)

seq_length = 24
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.9)
test_size = len(y) - train_size
dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

# Save minmax scaler

In [ ]:
import joblib
joblib.dump(sc, 'temperature_preproc.joblib')

# ML

In [ ]:
from temperature_lstm_model import LSTMModel

In [ ]:
num_epochs = 1500
learning_rate = 0.01

model = LSTMModel(len(trainX[0]))

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Prepare for mini-batching, using PyTorch's DataLoader
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y

from torch.utils.data import DataLoader

# Assuming you have 'data' and 'targets' as your input and target data, respectively.
dataset = CustomDataset(trainX, trainY)
batch_size = 1000  # You can set your desired batch size here.
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Train the model
for epoch in range(200):
    for batch_x, batch_y in data_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)

        # obtain the loss function
        loss = criterion(outputs, batch_y)
        loss.backward()
        
        optimizer.step()
    if epoch % 25 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
   

#### Testing that it does different things with different input

##### One-step ahead Prediction (does not work well!)

In [ ]:
from collections import deque

# Switch mode to evaluation
model.eval()

# Build up history by taking one slice of j last observations
j = 24
base_tensor = dataX[j:j+1]
base_list = base_tensor.data.numpy().tolist()[0]

# Set up deque-like cache (FIFO)
cache = deque([], maxlen=24)
for value in base_list:
    cache.append(value)

# Predict them one-step ahead
for i in range(24):
    prediction = model(torch.Tensor([cache]))
    # Update cache for new prediction
    cache.append([prediction[0][0].item()])

predictions = sc.inverse_transform(np.array(cache).reshape(-1, 1))

pre_prediction_values = sc.inverse_transform(np.array([np.mean(base_list) for x in predictions]).reshape(-1, 1))
predicted_values = np.concatenate((pre_prediction_values, np.array(predictions).reshape(-1, 1)))
plt.plot(predicted_values, c='r')

# True values
# Base slice on which predictions are made on
base_values = sc.inverse_transform(dataY[j].data.numpy().reshape(-1, 1))
# j*2 because j is the length of one slice and we want to check history
to_predict_values = sc.inverse_transform(dataY[j*2].data.numpy().reshape(-1, 1))
true_values = np.concatenate((base_values, to_predict_values))

plt.plot(true_values, c='b')
plt.rcParams['figure.figsize'] = [4, 4]
plt.show()

##### Multi-step (not one-step ahead) Prediction

In [ ]:
from collections import deque

# Switch mode to evaluation
model.eval()

# Build up history by taking one slice of j last observations
j = 24
base_tensor = dataX[j:j+1]

# Predict them all at once
prediction = model(torch.Tensor(base_tensor))

predictions = [sc.inverse_transform([[val.item()]])[0][0] for val in prediction[0]]

pre_prediction_values = sc.inverse_transform(np.array([np.mean(cache) for x in predictions]).reshape(-1, 1))
predicted_values = np.concatenate((pre_prediction_values, np.array(predictions).reshape(-1, 1)))
plt.plot(predicted_values, c='r')

# True values
# Base slice on which predictions are made on
base_values = sc.inverse_transform(dataY[j].data.numpy().reshape(-1, 1))
# j*2 because j is the length of one slice and we want to check history
to_predict_values = sc.inverse_transform(dataY[j*2].data.numpy().reshape(-1, 1))
true_values = np.concatenate((base_values, to_predict_values))

plt.plot(true_values, c='b')
plt.rcParams['figure.figsize'] = [8, 4]
plt.show()

##### Nice illustrative chart for train/test performance

In [ ]:
model.eval()
train_predict = model(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.suptitle('Time-Series Prediction')
plt.show()

In [ ]:
torch.save(model.state_dict(), "temperature_lstm.model") 

In [ ]:
new_model = LSTMModel()

In [ ]:
new_model.load_state_dict(torch.load("temperature_lstm.model"))

In [ ]:
new_model.eval()
train_predict = new_model(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

#plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot[:,0][3000:3024], c='b', label='Actual')
plt.plot(data_predict[:,0][3000:3024], c='r', label='Predicted')
plt.suptitle('Time-Series Prediction')
plt.legend()
plt.show()